# Project

## Data loading

### Create Parquet file Q1.1

In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

# Define paths
data_dir = "data"

pd.set_option('display.max_columns', None)

def read_patient_data(file_path):
    df = pd.read_csv(file_path, sep=",")
    return df

# Process each patient file
def process_patient_data(patient_file):
    df = read_patient_data(patient_file)

    df['Time'] = pd.to_timedelta(df['Time'] + ':00')
    df['Time'] = df['Time'].dt.ceil('h')
    df['Time'] = df['Time'].dt.total_seconds() // 3600
    
    return df

def generate_parquet(letter:str, keepICUType=False):

    folder_path = "data/set-" + letter
    static_columns = ["RecordID", "Age", "Gender", "Height", "ICUType", "Weight"] #ICU-Type can be dropped later
    df_processed = pd.DataFrame()

    for file_name in tqdm(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, file_name)
        df = process_patient_data(file_path)
        df = df.pivot_table(index="Time", columns="Parameter", values="Value", aggfunc="last").reset_index()
        df = df.reindex(columns= ["Time"] + static_columns + [col for col in df.columns if col not in static_columns + ["Time"]])
        df.set_index('Time', inplace=True)
        df = df.reindex(range(49), fill_value=np.nan)
        df[static_columns] = df[static_columns].ffill().bfill()
        df.reset_index(drop=False, inplace=True)
        df_processed = pd.concat([df_processed, df], ignore_index=True)

    if not keepICUType:
        df_processed = df_processed.drop(columns=["ICUType"])

    df_processed.to_parquet(f"data_{letter}_raw.parquet", engine="pyarrow", index=False)

    return df_processed



In [12]:
for letter in ["a", "b", "c"]:
    generate_parquet(letter)


100%|██████████| 4000/4000 [00:31<00:00, 128.70it/s]


### TBD: Generate nice Plots Q1.2

### Data imputation Q1.3

In [38]:
## Forward fill TODO: maybe later different imputation and fill values that never come up (mean value)
from sklearn.preprocessing import StandardScaler

for letter in ["a", "b", "c"]:
    ##Forwardfill and impute
    df = pd.read_parquet(f"data_{letter}_raw.parquet")
    df['Height'] = df['Height'].replace(-1.0, np.nan)
    df['Weight'] = df['Weight'].replace(-1.0, np.nan)
    df['Gender'] = df['Gender'].replace(-1.0, np.nan)
    df['NIDiasABP'] = df['NIDiasABP'].replace(-1.0, np.nan)
    df['DiasABP'] = df['DiasABP'].replace(-1.0, np.nan)
    df['Temp'] = df['Temp'].replace(-0.5, np.nan)
    df['Temp'] = df['Temp'].replace(-17.8, np.nan) # What to do with negative temp? Theoretically possible but not likely
    df.update(df.groupby('RecordID').ffill())

    ##Scale
    scaler = StandardScaler()
    columns_to_scale = ['Age', 'Height', 'Weight', 'BUN', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT', 'HR', 'K', 'Lactate', 'MAP', 'Mg', 'NIDiasABP', 'NIMAP', 'NISysABP', 'Na', 'PaCO2', 'PaO2', 'Platelets', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'Urine', 'WBC', 'pH', 'MechVent', 'TroponinT', 'ALP', 'ALT', 'AST', 'Albumin', 'Bilirubin', 'Cholesterol', 'TroponinI']
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])



    df.to_parquet(f"data_{letter}_cleaned.parquet", engine="pyarrow", index=False)

In [ ]:
df = pd.read_parquet(f"data_a_cleaned.parquet")
df.head()

,Time,RecordID,Age,Gender,Height,Weight,BUN,Creatinine,GCS,Glucose,HCO3,HCT,HR,K,Mg,NIDiasABP,NIMAP,NISysABP,Na,Platelets,RespRate,Temp,TroponinT,Urine,WBC,ALP,ALT,AST,Albumin,Bilirubin,FiO2,MechVent,PaCO2,PaO2,pH,DiasABP,MAP,SaO2,SysABP,Lactate,Cholesterol,TroponinI
0,0,132592.0,-1.665694,0.0,NaN,-0.444181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,132592.0,-1.665694,0.0,NaN,-0.444181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,132592.0,-1.665694,0.0,NaN,-0.444181,NaN,NaN,0.831874,NaN,NaN,NaN,1.425634,NaN,NaN,-0.941302,-0.494435,0.121859,NaN,NaN,0.422332,-0.314854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,132592.0,-1.665694,0.0,NaN,-0.444181,1.98615,0.633894,0.831874,8.374957,-2.854564,-1.215369,1.481737,1.946572,1.961486,-0.293028,0.020733,0.281725,0.232021,0.768255,0.243402,-0.314854,-0.349712,-0.064763,0.376511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,132592.0,-1.665694,0.0,NaN,-0.444181,1.98615,0.633894,0.831874,8.374957,-2.854564,-1.215369,1.425634,1.946572,1.961486,-0.617165,-0.323141,0.041925,0.232021,0.768255,0.780193,-0.314854,-0.349712,-0.382261,0.376511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
